In [2]:
import logging
 
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Log等级总开关
log_file_abs = "flask.log"
 
stream_handler = logging.StreamHandler()  # 日志控制台输出
 
handler = logging.FileHandler(log_file_abs, mode='w', encoding='UTF-8')  # 日志文件输出
handler.setLevel(logging.DEBUG)
 
# 控制台输出格式
stream_format = logging.Formatter("Time: %(asctime)s -- INFO: %(message)s")
 
# 文件输出格式
logging_format = logging.Formatter("Time: %(asctime)s -- INFO: %(message)s")
 
handler.setFormatter(logging_format)  # 为改处理器handler 选择一个格式化器
stream_handler.setFormatter(stream_format)
 
logger.addHandler(handler)  # 为记录器添加 处理方式Handler
logger.addHandler(stream_handler)
 
logger.info("------logger.info-----")

Time: 2023-02-28 14:50:32,127 -- INFO: ------logger.info-----
Time: 2023-02-28 14:50:32,127 -- INFO: ------logger.info-----


In [14]:
import time
print(time.localtime())

time.struct_time(tm_year=2023, tm_mon=3, tm_mday=1, tm_hour=13, tm_min=40, tm_sec=7, tm_wday=2, tm_yday=60, tm_isdst=0)


In [3]:
print("{:.6f} c".format(1))


1.000000 c


In [3]:
from model.transunet import TransUNet
import torch
vit = TransUNet(img_dim=512,
                          in_channels=3,
                          out_channels=128,
                          head_num=4,
                          mlp_dim=512,
                          block_num=8,
                          patch_dim=16,
                          class_num=2)
# print(sum(p.numel() for p in vit.parameters()))
print(vit(torch.rand(1, 3, 512, 512)).shape)


67865730
torch.Size([1, 2, 512, 512])
